# Colab Templates & Roadmap

This short notebook is a companion to the project checkpoint. It contains:

- A concise, Colab-focused roadmap (milestones + success criteria).
- Small, safe example cells you can run in Colab or locally to bootstrap work: DRIVE mount + `DRIVE_ROOT` setup, `compose_flow` demo, `preview_url` dry-run, and a `queue` demo.

Milestones:
1. Stabilize developer onboarding: reliable DRIVE_ROOT detection and minimal example flows (this notebook).
2. Add small unit-tests for `composer.compose_flow` and `queue` operations (CI-friendly).
3. Provide polished notebook README + troubleshooting for Colab/Drive mounts.

Success criteria:
- A newcomer can open this notebook, run the first 3 cells, and create a flow JSON under Drive.
- Examples are safe (no large downloads or secrets) and demonstrate the standard import pattern for `Helper Scipts` modules.

In [ ]:
# Cell 1 — DRIVE_ROOT detection and (optional) Colab mount
from pathlib import Path
import os

# Default DRIVE_ROOT used across helper modules. In Colab this will be '/content/drive/MyDrive/ComfyUI'
DRIVE_ROOT = Path(os.environ.get('DRIVE_ROOT', '/content/drive/MyDrive/ComfyUI'))
print('Initial DRIVE_ROOT:', DRIVE_ROOT)

# If running inside Google Colab, mount Drive (safe: only if you want to).
try:
code
python
# Cell 5 — Cloudflared & ComfyUI start/restart helpers (actionable examples)
# The helper functions remain safe, but the usage examples below call them with dry_run=False so they are actionable.
# In Colab you should background cloudflared with nohup and redirect logs; examples are provided.
import subprocess
import os
from pathlib import Path

def run_cmd(cmd, dry_run=True):
    print('CMD:', cmd)
    if dry_run:
        print('[dry-run] not executing')
        return None
    out = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, text=True)
    print(out)
    return out

def start_cloudflared(tunnel_name=None, port=8188, dry_run=True, background=False):
    # If cloudflared is already installed, start or run a tunnel without reinstalling.
    if tunnel_name:
        base = f'cloudflared tunnel run {tunnel_name} --url http://localhost:{port}'
    else:
        base = f'cloudflared tunnel --url http://localhost:{port}'
    if background:
        log = '/tmp/cloudflared.log'
        cmd = f'nohup {base} >{log} 2>&1 &'
        print('Starting cloudflared in background; logs ->', log)
    else:
        cmd = base
    return run_cmd(cmd, dry_run=dry_run)

def restart_cloudflared(dry_run=True, background=False, port=8188):
    # Best-effort restart: kill existing cloudflared and re-run. Does not reinstall anything.
    cmds = [
        'pkill -f cloudflared || true',
        'sleep 0.5',
    ]
    out = []
    for c in cmds:
        out.append(run_cmd(c, dry_run=dry_run))
    # start again
    out.append(start_cloudflared(port=port, dry_run=dry_run, background=background))
    return out

def restart_comfyui(process_name_keyword='ComfyUI', restart_command=None, dry_run=True):
    # Attempt to gracefully restart ComfyUI without reinstalling. If COMFYUI_LAUNCH_CMD is set, it will be used to relaunch.
    try:
        ps = subprocess.check_output('ps aux', shell=True, text=True)
        lines = [l for l in ps.splitlines() if process_name_keyword in l and 'grep' not in l]
        print('Found ComfyUI processes:', len(lines))
        run_cmd('pkill -f 
 || true', dry_run=dry_run)
        if restart_command:
            return run_cmd(restart_command, dry_run=dry_run)
        launch = os.environ.get('COMFYUI_LAUNCH_CMD')
        if launch:
            print('Using COMFYUI_LAUNCH_CMD from environment to restart')
            return run_cmd(launch, dry_run=dry_run)
        print('No launch command found; pkilled only')
        return 'pkilled_only'
    except subprocess.CalledProcessError as e:
        print('Error while inspecting processes', e)
        return None

# Actionable usage examples (UNCOMMENT to run in Colab):
# Start cloudflared in background (logs at /tmp/cloudflared.log):
# start_cloudflared(port=8188, dry_run=False, background=True)
# Restart cloudflared (actionable):
# restart_cloudflared(dry_run=False, background=True, port=8188)
# Restart ComfyUI using an environment launch command: set os.environ['COMFYUI_LAUNCH_CMD'] then:
# restart_comfyui(dry_run=False)

print('Cloudflared & ComfyUI helper functions defined. Examples below are ready to run (set dry_run=False).')
print('In Colab: to keep cloudflared running after the cell completes, use background=True and nohup (example provided).')
    print('downloader module loaded — streaming_download exists:', hasattr(dl, 'streaming_download'))
    # Example usage (commented):
    # dl.streaming_download(url, str(DRIVE_ROOT / 'models' / 'example.safetensors'), verify_sha256=None)
else:
    print('downloader.py not found')

In [ ]:
# Cell 4 — Queue demo (enqueue, dequeue, mark_done)
queue_path = Path('Helper Scipts') / 'queue.py'
if queue_path.exists():
    qmod = import_module_from_path('queue', queue_path)
    # Create a small demo item and push it to the sqlite queue stored under DRIVE_ROOT/state/queue.db
    demo_item = {'task': 'demo', 'payload': {'prompt': 'test prompt'}}
    try:
        id_ = qmod.enqueue(demo_item)
        print('Enqueued id', id_)
        front = qmod.dequeue()
        print('Dequeued:', front)
        if front and 'id' in front:
            qmod.mark_done(front['id'])
            print('Marked done for id', front['id'])
    except Exception as e:
        print('Queue demo error:', e)
else:
    print('queue.py not found')

Notes and next steps:

- These cells are intentionally conservative: they demonstrate how to import helper modules from the `Helper Scipts` folder (which contains a space in its name) and exercise non-destructive functionality (create a small flow JSON, enqueue/dequeue a demo item).
- Next: we should add one or two unit tests that assert `compose_flow` returns the expected node properties and that queue operations round-trip. I can add a minimal `tests/test_basics.py` and update CI to run it.
- If you want, I can convert these cells into a short PR that adds a notebook README and tests. Tell me which milestone to do next (Tests, Docs, Delivery).

In [ ]:
# Cell 5 — Cloudflared & ComfyUI start/restart helpers (dry-run by default)
# This cell provides helper functions to start or restart a cloudflared tunnel and restart a running ComfyUI process
# without reinstalling dependencies or re-downloading Docker images. It is safe by default (dry_run=True).
import subprocess
import os
from pathlib import Path

def run_cmd(cmd, dry_run=True):
    print('CMD:', cmd)
    if dry_run:
        print('[dry-run] not executing')
        return None
    return subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, text=True)

def start_cloudflared(tunnel_name=None, port=8188, dry_run=True):
    # If cloudflared is already running, this will not reinstall it — it will attempt to start or restart the tunnel.
    # tunnel_name: optional named tunnel; if None, use local http proxy mode.
    if tunnel_name:
        cmd = f'cloudflared tunnel run {tunnel_name} --url http://localhost:{port}'
    else:
        # run a quick local tunnel proxy on the desired port (ensure cloudflared is installed)
        cmd = f'cloudflared tunnel --url http://localhost:{port}'
    return run_cmd(cmd, dry_run=dry_run)

def restart_cloudflared(dry_run=True):
    # Best-effort: try to kill cloudflared and then start it again (system dependent).
    # This avoids re-installing packages or re-creating images.
    # On Colab you may need to run a background process (nohup) — keep this simple here.
    cmds = [
        'pkill -f cloudflared || true',
        'sleep 0.5',
        